<a href="https://colab.research.google.com/github/rkeaveney/ped-detect-predict/blob/main/yolo_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!tar -xzf /content/drive/MyDrive/UL/EE6061/data/all_data.tar.gz

In [ ]:
%%capture
!git clone https://github.com/WongKinYiu/yolov7.git yolov7 
!pip install -r yolov7/requirements.txt

In [ ]:
import torch
from IPython.display import display, Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
#from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pickle

random.seed(108)

In [ ]:
images = [os.path.join("/content/images", x) for x in os.listdir("/content/images/")]
annotations = [os.path.join("/content/yolo_labels", x) for x in os.listdir("/content/yolo_labels/")]

images.sort()
annotations.sort()

train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
"""with open('train_images.pickle', 'wb') as f:
    pickle.dump(train_images, f)
with open('train_annotations.pickle', 'wb') as f:
    pickle.dump(train_annotations, f)

with open('val_images.pickle', 'wb') as f:
    pickle.dump(val_images, f)
with open('val_annotations.pickle', 'wb') as f:
    pickle.dump(val_annotations, f)

with open('test_images.pickle', 'wb') as f:
    pickle.dump(test_images, f)
with open('test_annotations.pickle', 'wb') as f:
    pickle.dump(test_annotations, f)

!cp *.pickle /content/drive/MyDrive/UL/EE6061/pickles/yolo"""

In [ ]:
!mkdir /content/images/train /content/images/val /content/images/test /content/labels/train /content/labels/val /content/labels/test

In [ ]:
"""import pickle as pkl

!mkdir pickles
!cp /content/drive/MyDrive/UL/EE6061/pickles/yolo/* pickles

pickles = [os.path.join("pickles", x) for x in os.listdir("pickles")]
for pickle in pickles:
    var = pickle.replace('pickles/', '').replace('.pickle', '')
    with open(pickle, 'rb') as f:
        exec(var + " = pkl.load(f)")

for im in train_images:
    la = im[:9] + 'yolo_labels' + im[15:26] + '.txt'
    os.rename(im, im[:16] + 'train/' + im[16:])
    os.rename(la, la[:9] + 'labels/train/' + la[21:])

for im in test_images:
    la = im[:9] + 'yolo_labels' + im[15:26] + '.txt'
    os.rename(im, im[:16] + 'test/' + im[16:])
    os.rename(la, la[:9] + 'labels/test/' + la[21:])

for im in val_images:
    la = im[:9] + 'yolo_labels' + im[15:26] + '.txt'
    os.rename(im, im[:16] + 'val/' + im[16:])
    os.rename(la, la[:9] + 'labels/val/' + la[21:])"""

In [ ]:
def safe_copy(file_path, out_dir, dst = None):
    """Safely copy a file to the specified directory. If a file with the same name already 
    exists, the copied file name is altered to preserve both.

    :param str file_path: Path to the file to copy.
    :param str out_dir: Directory to copy the file into.
    :param str dst: New name for the copied file. If None, use the name of the original
        file.
    """
    name = dst or os.path.basename(file_path)
    if not os.path.exists(os.path.join(out_dir, name)):
        shutil.copy(file_path, os.path.join(out_dir, name))
    else:
        base, extension = os.path.splitext(name)
        i = 1
        while os.path.exists(os.path.join(out_dir, '{}_{}{}'.format(base, i, extension))):
            i += 1
        shutil.copy(file_path, os.path.join(out_dir, '{}_{}{}'.format(base, i, extension)))

In [ ]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            safe_copy(f, destination_folder)
        except:
            print(f)
            assert False

In [ ]:
labels = [os.path.join("/content/labels/test/", x) for x in os.listdir("/content/labels/test/")]
for label in labels:
    with open(label, 'r') as f:
        lines = f.readlines()
    with open(label, 'w') as f:
        for line in lines:
            x = line.split()
            if x[0] != '0':
                f.write(line)
            elif float(x[4]) > 100/1920:
                f.write(line)

In [ ]:
move_files_to_folder(train_images, '/content/images/train')
move_files_to_folder(val_images, '/content/images/val/')
move_files_to_folder(test_images, '/content/images/test/')
move_files_to_folder(train_annotations, '/content/labels/train/')
move_files_to_folder(val_annotations, '/content/labels/val/')
move_files_to_folder(test_annotations, '/content/labels/test/')

In [ ]:
!rm /content/images/*.jpg
!rm /content/labels/*.txt

rm: cannot remove '/content/images/*.jpg': No such file or directory


In [ ]:
!cp /content/drive/MyDrive/UL/EE6061/yaml/data.yaml /content/yolov7/data/

In [ ]:
%%capture
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt -P /content/yolov7/
!cp /content/drive/MyDrive/UL/EE6061/yaml/yolov7_ped_detect.yaml /content/yolov7/cfg/training/

In [ ]:
!cat /content/yolov7/data/data.yaml

train: /content/images/train/ 
val:  /content/images/val/
test: /content/images/test/

# number of classes
nc: 4

# class names
names: ['pedestrian', 'cyclist', 'vehicle', 'sign']

In [ ]:
!python /content/yolov7/train.py --img-size 640 --cfg /content/yolov7/cfg/training/yolov7_ped_detect.yaml --hyp /content/yolov7/data/hyp.scratch.custom.yaml --batch 10 --epochs 75 --data /content/yolov7/data/data_1class.yaml --weights /content/yolov7/yolov7_training.pt --workers 24 --name yolo_road_det

YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536.125MB)

Namespace(adam=False, artifact_alias='latest', batch_size=10, bbox_interval=-1, bucket='', cache_images=False, cfg='/content/yolov7/cfg/training/yolov7_ped_detect.yaml', data='/content/yolov7/data/data_1class.yaml', device='', entity=None, epochs=75, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='/content/yolov7/data/hyp.scratch.custom.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolo_road_det', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolo_road_det', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=10, upload_dataset=False, v5_metric=False, weights='/content/yolov7/yolov7_training.pt', workers=24, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:600

In [ ]:
!mkdir /content/drive/MyDrive/UL/EE6061/runs/train_$(date "+%Y.%m.%d_%H:%M")
!cp -r /content/runs/train/* /content/drive/MyDrive/UL/EE6061/runs/train_$(date "+%Y.%m.%d_%H:%M")

In [ ]:
!mkdir runs runs/train
!cp -r /content/drive/MyDrive/UL/EE6061/runs/train/train_v7/* runs/train

In [ ]:
!ls runs/train/yolo_road_det/weights/

best.pt       epoch_024.pt  epoch_070.pt  epoch_072.pt	epoch_074.pt  last.pt
epoch_000.pt  epoch_049.pt  epoch_071.pt  epoch_073.pt	init.pt


In [ ]:
!python /content/yolov7/test.py --data /content/yolov7/data/data.yaml --img-size 640 --batch 10 --weights runs/train/yolo_road_det/weights/best.pt --conf 0.367 --task test --iou 0.6 --name ped_detect_test

Namespace(weights=['runs/train/yolo_road_det/weights/best.pt'], data='/content/yolov7/data/data.yaml', batch_size=10, img_size=640, conf_thres=0.367, iou_thres=0.6, task='test', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='ped_detect_test', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36497954 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Trac

In [ ]:
!mkdir /content/drive/MyDrive/UL/EE6061/runs/test_$(date "+%Y.%m.%d_%H:%M")
!cp -r /content/runs/test/* /content/drive/MyDrive/UL/EE6061/runs/test_$(date "+%Y.%m.%d_%H:%M")

In [ ]:
!cp /content/drive/MyDrive/UL/EE6061/images/pedestrians.mp4 .

In [ ]:
!python /content/yolov7/detect.py --source images/test --weights runs/train/yolo_road_det/weights/best.pt --conf 0.367 --name yolo_road_det --save-txt

Streaming output truncated to the last 5000 lines.
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0046.jpg
7 vehicles, Done. (23.1ms) Inference, (1.0ms) NMS
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0057.jpg
7 vehicles, Done. (23.1ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0058.jpg
6 vehicles, Done. (23.1ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0065.jpg
4 vehicles, Done. (23.1ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0075.jpg
3 vehicles, Done. (23.0ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0094.jpg
2 vehicles, Done. (23.1ms) Inference, (1.1ms) NMS
 The image with the result is saved in: runs/detect/yolo_road_det/s0070f0112.jpg
2 vehicles, Done. (23.1ms) Inference, (1.1ms) NMS
 The image with the result is sa